In [1]:
#!/usr/bin/python
#NN_option Pricing.ipynb
#Parth Parakh
#purpose: Coursework

#---------------------------------------------------------------------
#import modules
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import fft
from scipy import interpolate
from scipy.interpolate import CubicSpline
import sklearn
from sklearn.model_selection import train_test_split 
#----------------------------------------------------------------------    
def BSM(price,strike,rate,time,v):
    
    S0 = price;
    K = strike;
    sigma = v;
    r = rate;
    T = time;
    q = 0;
    #Analytical Solution
    d1 = ((np.log(S0/K)+(r-q+0.5*sigma**2)*T))/(sigma*np.sqrt(T));
    # d2 = d1 - sigma*sqrt(T);
    d2 = ((np.log(S0/K)+(r-q-0.5*sigma**2)*T))/(sigma*np.sqrt(T));
    Vca = (S0*np.exp(-q*T)*norm.cdf(d1)) - (K*np.exp(-r*T)*norm.cdf(d2));
    # Put-call parity: V_call + K*exp(-r*T) = V_put - S_0
    Vpa = (K*np.exp(-r*T)*norm.cdf(-d2)) - (S0*np.exp(-q*T)*norm.cdf(-d1));
    
    return Vca;
#----------------------------------------------------------------------
#Defining the parameters

s = 115;
r = 0.05;
v = 0.5;

p = np.linspace(75,130,1000);
t = np.linspace(0,1,100);

time, price = np.meshgrid(t,p);

call = np.zeros((len(price),len(price[0])))

for i in range(len(price)):
    for j in range(len(price[0])):
        call[i][j] = BSM(price[i][j],s,r,time[i][j],v);

/Users/parthparakh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars
/Users/parthparakh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_scalars


In [2]:
Y= call.flatten()
#Y = Y.reshape((1,len(Y)))
time = time.flatten()
price = price.flatten()
X =  np.vstack((time,price)).T

In [3]:
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size=0.3,random_state=2)
#XTrain = XTrain.T
#XTest = XTest.T
#YTrain = YTrain.T
#YTest = YTest.T

np.shape(XTest)

(30000, 2)

In [4]:
# Define useful functions    

# Activation function
def sigmoid(t):
    return 1/(1+np.exp(-t))

# Derivative of sigmoid
def sigmoid_derivative(p):
    return p * (1 - p)

# Class definition
class NeuralNetwork:
    def __init__(self, x,y):
        self.input = x
        self.weights1= np.random.rand(self.input.shape[1],10) # considering we have 4 nodes in the hidden layer
        self.weights2 = np.random.rand(10,1)
        self.y = y
        self.output = np. zeros(y.shape)
        
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.layer2 = sigmoid(np.dot(self.layer1, self.weights2))
        return self.layer2
        
    def backprop(self):
        d_weights2 = np.dot(self.layer1.T, 2*(self.y -self.output)*sigmoid_derivative(self.output))
        d_weights1 = np.dot(self.input.T, np.dot(2*(self.y -self.output)*sigmoid_derivative(self.output), self.weights2.T)*sigmoid_derivative(self.layer1))
    
        self.weights1 += d_weights1
        self.weights2 += d_weights2

    def train(self, X, y):
        self.output = self.feedforward()
        self.backprop()
         

In [ ]:
NN = NeuralNetwork(X,Y)
for i in range(1000): # trains the NN 1,000 times
    NN.train(XTrain,YTrain)